In [1]:
%%bash
wget --no-check-certificate 'https://drive.usercontent.google.com/download?id=1qe5hOSBxzIuxBb1G_Ih5X-O65QElollE&export=download&confirm=t&uuid=b2002093-cc6e-4bd5-be47-9603f0b33470
' -O KuaiRec.zip
unzip KuaiRec.zip -d data_final_project

--2025-04-14 11:29:41--  https://drive.usercontent.google.com/download?id=1qe5hOSBxzIuxBb1G_Ih5X-O65QElollE&export=download&confirm=t&uuid=b2002093-cc6e-4bd5-be47-9603f0b33470%0A


Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.179.65, 2a00:1450:4007:813::2001
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.179.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 431964858 (412M) [application/octet-stream]
Saving to: ‘KuaiRec.zip’

     0K .......... .......... .......... .......... ..........  0% 3.70M 1m51s
    50K .......... .......... .......... .......... ..........  0% 12.7M 72s
   100K .......... .......... .......... .......... ..........  0% 9.69M 62s
   150K .......... .......... .......... .......... ..........  0% 18.1M 52s
   200K .......... .......... .......... .......... ..........  0% 9.05M 51s
   250K .......... .......... .......... .......... ..........  0% 32.2M 44s
   300K .......... .......... .......... .......... ..........  0% 7.62M 46s
   350K .......... .......... .......... .......... ..........  0% 11.0M 45s
   400K .......... .......... .....

Archive:  KuaiRec.zip
   creating: data_final_project/KuaiRec 2.0/
  inflating: data_final_project/KuaiRec 2.0/LICENSE  
  inflating: data_final_project/KuaiRec 2.0/Statistics_KuaiRec.ipynb  
   creating: data_final_project/KuaiRec 2.0/data/
  inflating: data_final_project/KuaiRec 2.0/data/big_matrix.csv  
  inflating: data_final_project/KuaiRec 2.0/data/item_categories.csv  
  inflating: data_final_project/KuaiRec 2.0/data/item_daily_features.csv  
  inflating: data_final_project/KuaiRec 2.0/data/kuairec_caption_category.csv  
  inflating: data_final_project/KuaiRec 2.0/data/small_matrix.csv  
  inflating: data_final_project/KuaiRec 2.0/data/social_network.csv  
  inflating: data_final_project/KuaiRec 2.0/data/user_features.csv  
   creating: data_final_project/KuaiRec 2.0/figs/
  inflating: data_final_project/KuaiRec 2.0/figs/KuaiRec.png  
  inflating: data_final_project/KuaiRec 2.0/figs/colab-badge.svg  
  inflating: data_final_project/KuaiRec 2.0/loaddata.py  


In [1]:
# Import libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.layers import Lambda
import ast

2025-04-21 01:08:12.534166: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Load Datasets

In [2]:
# Drop every time related columns except "timestamp" because of redundancy (keeping timestamp because it is UTC). Same for the play time and video time
train_set = pd.read_csv("data_final_project/KuaiRec 2.0/data/big_matrix.csv", usecols=["user_id", "video_id", "watch_ratio"], skiprows=range(1, 5000000)).reset_index(drop=True)
test_set = pd.read_csv("data_final_project/KuaiRec 2.0/data/small_matrix.csv", usecols=["user_id", "video_id", "watch_ratio"]).reset_index(drop=True)

tags = pd.read_csv("data_final_project/KuaiRec 2.0/data/item_categories.csv").reset_index(drop=True)

In [3]:
train_set["watch_ratio"] = np.clip(train_set["watch_ratio"], 0, 3)

ratio_scaler = MinMaxScaler(feature_range=(-1, 1))
train_set[["watch_ratio"]] = ratio_scaler.fit_transform(train_set[["watch_ratio"]])

tags["feat"] = tags["feat"].apply(ast.literal_eval)

In [4]:
def clean_df(df: pd.DataFrame):
    df = df.dropna()
    df = df.drop_duplicates()
    #df = df[df["timestamp"] >= 0]
    df = df.drop(df[df["watch_ratio"] == 0].index)
    return df

train_set = clean_df(train_set)
test_set = clean_df(test_set)

## Build users features

In [5]:
def build_user_features():
    users_features = train_set.merge(tags, on="video_id").explode("feat")
    users_features = users_features.groupby(['user_id', 'feat'])['watch_ratio'].mean().reset_index()
    users_features = users_features.pivot(index='user_id', columns='feat', values='watch_ratio').fillna(0)
    users_features.columns = [f'tag_{int(col)}' for col in users_features.columns]
    users_features = users_features.reset_index()
    return users_features
users_features = build_user_features()
user_scaler = MinMaxScaler(feature_range=(-1, 1))
users_features[users_features.columns[1:]] = user_scaler.fit_transform(users_features[users_features.columns[1:]])

In [6]:
users_features

,user_id,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,...,tag_21,tag_22,tag_23,tag_24,tag_25,tag_26,tag_27,tag_28,tag_29,tag_30
0,2881,0.000000,0.685631,-0.003714,0.000000,0.000000,0.710057,1.000000,0.147110,0.754560,...,0.000000,0.000000,0.000000,0.000000,0.441569,0.823617,0.000000,0.699098,0.000000,0.000000
1,2882,-0.706894,-0.211011,-0.647215,-0.894992,-0.549380,-0.207431,-0.330913,-0.425758,-0.110919,...,0.000000,-0.908443,-0.921295,-0.390548,-0.425091,-0.251617,-0.611571,-0.177075,-0.710186,-0.833272
2,2883,-0.275961,-0.275272,-0.761695,-0.485713,-0.132249,0.022043,-0.179249,-0.524321,-0.072623,...,-0.832333,0.000000,0.000000,0.000000,-0.272483,-0.117837,0.272651,-0.167925,0.000000,-0.648186
3,2884,-0.801035,-0.098332,-0.685825,-0.471127,-0.470524,0.009574,-0.353259,-0.479314,0.217721,...,-0.818040,0.000000,0.000000,-0.864667,0.157822,-0.212546,-0.583240,-0.133130,0.000000,-0.267076
4,2885,-0.284938,0.417878,-0.317576,-0.170532,-0.181905,0.570633,0.283889,0.003298,0.482364,...,-0.313524,-0.901373,-0.288993,0.000000,0.051590,0.407665,0.027710,0.403527,-0.392283,-0.282164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4290,7171,-0.651365,-0.034104,-0.588513,-0.648916,-0.531984,-0.234991,-0.323095,-0.353380,-0.229107,...,-0.813909,-0.987019,-0.722307,-0.956888,-0.249169,-0.120259,-0.435685,0.072351,-0.878724,-0.659023
4291,7172,-0.525270,0.146116,-0.512970,-0.772479,-0.552969,-0.048811,0.092660,-0.284452,0.202429,...,-0.072054,-0.841626,-0.129957,-0.945667,-0.198665,0.125225,-0.468532,0.263433,0.000000,-0.669179
4292,7173,-0.804293,-0.601961,-0.415235,0.000000,-0.598412,-0.590164,-0.290524,-0.574849,-0.424885,...,0.000000,-0.923078,-0.705670,0.000000,-0.299500,-0.124716,0.000000,-0.094553,0.000000,-0.285723
4293,7174,-0.285731,-0.260025,-0.634800,-0.552098,-0.395112,-0.111568,-0.332651,-0.388751,-0.139064,...,-0.546474,-0.944233,0.287876,-0.918607,-0.302582,-0.212243,-0.568218,-0.018081,-0.555975,-0.484958


## Build items features

In [7]:
def build_item_features():
    items_features = tags.explode("feat")
    items_features = pd.crosstab(items_features["video_id"], items_features["feat"])
    items_features.columns = [f"tag_{int(col)}" for col in items_features.columns]
    items_features = items_features.reset_index()
    return items_features
items_features = build_item_features()

item_scaler = MinMaxScaler(feature_range=(-1, 1))
items_features[items_features.columns[1:]] = item_scaler.fit_transform(items_features[items_features.columns[1:]])

In [8]:
items_features

,video_id,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,...,tag_21,tag_22,tag_23,tag_24,tag_25,tag_26,tag_27,tag_28,tag_29,tag_30
0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0
2,2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,3,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0
4,4,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10723,10723,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
10724,10724,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
10725,10725,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
10726,10726,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [9]:
user_train = train_set.drop(columns=["video_id", "watch_ratio"]).merge(build_user_features(), on="user_id")
user_train

KeyboardInterrupt: 

In [7]:
item_train = train_set.drop(columns=["user_id", "watch_ratio"]).merge(build_item_features(), on="video_id")
item_train

,video_id,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,...,tag_21,tag_22,tag_23,tag_24,tag_25,tag_26,tag_27,tag_28,tag_29,tag_30
0,3649,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9598,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,5262,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1963,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,8234,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11331048,1281,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
11331049,3407,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11331050,10360,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
11331051,10360,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [11]:
y_train = train_set["watch_ratio"].to_list()
y_train

[1.2733965215790926,
 1.2440823015294975,
 0.1076125442589782,
 0.0898852971845672,
 0.078,
 1.572294776119403,
 0.1753976030752996,
 2.212061894108874,
 0.1304918032786885,
 1.4062659977475171,
 1.8889678703440431,
 2.0970967741935485,
 2.1577385857919893,
 0.0679368262722486,
 1.2918181818181818,
 0.0902172714238447,
 1.935958459541324,
 33.27602070155262,
 1.4420652173913044,
 0.0716965742251223,
 1.443190132747988,
 0.2224569876245095,
 0.0780696851127259,
 2.788442109517074,
 0.0871412288881981,
 0.065748031496063,
 0.0811320754716981,
 0.0772630834512022,
 0.1113266097750193,
 1.2807249054279932,
 2.345526524148852,
 1.15256,
 0.1285643645953519,
 1.3400613896381732,
 0.1105194805194805,
 1.5229745431498891,
 2.666480342835849,
 0.1356666666666666,
 0.0922003183543528,
 0.1092578226014998,
 2.3708835996087383,
 2.403380841895923,
 2.530868503662365,
 0.0999280230326295,
 0.0617862621441963,
 2.074151948416036,
 1.5030927835051546,
 2.1438668826387866,
 0.1294600568361225,
 1.7442

## No ram oops

In [9]:
# Pre-index user and video features using dictionaries
user_features_dict = users_features.set_index("user_id").to_dict(orient="index")
video_features_dict = items_features.set_index("video_id").to_dict(orient="index")

# Build dataset without re-filtering or full joins
dataset = []
for row in train_set.itertuples(index=False):
    user_feats = user_features_dict.get(row.user_id)
    video_feats = video_features_dict.get(row.video_id)
    if user_feats is not None and video_feats is not None:
        dataset.append([
            list(user_feats.values()),
            list(video_feats.values()),
            row.watch_ratio
        ])

In [10]:
user_train = np.array([row[0] for row in dataset])
item_train = np.array([row[1] for row in dataset])
y_train = np.array([row[2] for row in dataset])

In [11]:
num_outputs = 1
tf.random.set_seed(1)

user_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_outputs),
    ### END CODE HERE ###
])

item_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_outputs),
    ### END CODE HERE ###
])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(len(user_train[0]),))
vu = Lambda(lambda x: tf.linalg.l2_normalize(x, axis=1))(user_NN(input_user))

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(len(item_train[0]),))
vm = Lambda(lambda x: tf.linalg.l2_normalize(x, axis=1))(item_NN(input_item))

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 31)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 31)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 1)         │     41,217 │ input_layer[0][0] │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_1        │ (None, 1)         │     41,217 │ input_layer_2[0]… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 1)         │          0 │ sequential[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 1)         │          0 │ sequential_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot (Dot)           │ (None, 1)         │          0 │ lambda[0][0],     │
│                     │                   │            │ lambda_1[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 82,434 (322.01 KB)

 Trainable params: 82,434 (322.01 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn,
              metrics=["mae"],)

In [ ]:
tf.random.set_seed(1)
model.fit(
    [user_train, item_train],
    y_train,
    epochs=50,
    batch_size=256,
    validation_split=0.4,
)

In [12]:
train_set['interaction'] = train_set['watch_ratio'].apply(lambda x: 1 if x > 1.0 else 0)
test_set['interaction'] = test_set['watch_ratio'].apply(lambda x: 1 if x > 1.0 else 0)